<a href="https://colab.research.google.com/github/a-massive-world/organ-segmentation/blob/main/thesis_stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#####CTRL+F and search for 'CTRL+START', to go to the starting cell block.


from IPython.display import clear_output as clear,Image
import json,os,time,tarfile,shutil,numpy as np, nibabel
from glob import glob

from nibabel import load,Nifti1Image,save
from tqdm import tqdm
from matplotlib import pyplot as plt
##############JUST PLOTTING

try:
  from humanfriendly import format_timespan, format_size
except ModuleNotFoundError:
  !pip install humanfriendly
  from humanfriendly import format_timespan, format_size


try:
  import pydicom
except ModuleNotFoundError:
  !pip install pydicom
  import pydicom


try:
  import dicom2nifti
except ModuleNotFoundError:
  !pip install dicom2nifti
  import dicom2nifti
  

try:
  import SimpleITK as sitk
except ModuleNotFoundError:
  !pip install simpleitk
  import SimpleITK as sitk


try:
  import wget
except ModuleNotFoundError:
  !pip install wget
  import wget

try:
  import monai
  from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
  )
  from monai.data import DataLoader, Dataset, CacheDataset
  from monai.utils import set_determinism
  from monai.utils import first

except ModuleNotFoundError:
  !pip install monai
  !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
  from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
  from monai.data import DataLoader, Dataset, CacheDataset
  from monai.utils import set_determinism
  from monai.utils import first



clear()

In [ ]:
!nvidia-smi

In [ ]:
def to_json(dic, name, string):
  # print('to_json:',string,sep,name)c
  path = main_path
  a = 1
  if string == 'write':
    
    if dic == {}:
      print('Given Dictionary is empty. Check again!!')
      return False
    if a == 1:  

      with open(path+name+'.json','w') as fp:
        json.dump(dic,fp)
    else:
      print('Did not save anything.')
      return False
    
    # print('saved '+name+'.json')
    print('saved',sep,name,'('+getfilesize(path+name+'.json',1)+')')

  if string == 'read':
    with open(path+name+'.json','r') as fp:
      data = json.load(fp)
    
    return data


def getfilesize(the_file,the_type):
  file_size = os.path.getsize(the_file)
  actual_file_size = format_size(file_size)

  la_output = the_file.split('/')[-1]+sep+actual_file_size
  
  if the_type not in empty_list:
    if the_type == 1:
      return la_output.split(sep)[-1]
    else:
      return file_size
  else:
    print(la_output)

def get_dir_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    
    return total



In [ ]:
separator = ' —— '
sep = ' — '
empty_list = ['',""," ",' ']

In [ ]:
main_path = '/content/drive/MyDrive/thesis_dataset/'
livpath = main_path+'Liver/Task03_Liver/'
mypath = main_path+'test_path/'

In [ ]:
# database = to_json({},'Liver/Task03_Liver/dataset','read')


In [ ]:
import os

def listdir_nohidden(path):
  for f in os.listdir(path):
    if not f.startswith('.'):
      yield f

def convertNsave(arr,file_dir, index=0):
    
  dicom_file = pydicom.dcmread(main_path+'sample_dicom/dcmimage.dcm')
  arr = arr.astype('uint16')
  dicom_file.Rows = arr.shape[0]
  dicom_file.Columns = arr.shape[1]
  dicom_file.PhotometricInterpretation = "MONOCHROME2"
  dicom_file.SamplesPerPixel = 1
  dicom_file.BitsStored = 16
  dicom_file.BitsAllocated = 16
  dicom_file.HighBit = 15
  dicom_file.PixelRepresentation = 1
  dicom_file.PixelData = arr.tobytes()
  dicom_file.save_as(os.path.join(file_dir, f'slice{index}.dcm'))

def nifti2dicom_1file(nifti_dir, out_dir):


  nifti_file = nibabel.load(nifti_dir)
  nifti_array = nifti_file.get_fdata()
  number_slices = nifti_array.shape[2]
  print('INPUT: ',nifti_dir)
  print('outPUT: ',out_dir)

  for slice_ in tqdm(range(number_slices)):
    convertNsave(nifti_array[:,:,slice_], out_dir, slice_)

def nifti2dicom_mfiles(file,out_dir=''):

  # files = listdir_nohidden(nifti_dir)
  # c = 0
  # for file in files:
    # clear()
  # print(str(c+1))
  
  print('Extracting',sep,file+'\n')
  # in_path = os.path.join(nifti_dir, file.split('/')[-1])
  out_path = os.path.join(out_dir, file.split('/')[-1])
  # if os.path.exists(out_path) is not True:
  #   os.mkdir(out_path)
  nifti2dicom_1file(file, out_path)
    
    # c+=1

      
  

In [ ]:
def slices_to_folder(patient,group_val,cluster,g64,dcom_folder):
# patient = the_name_of_the_patient
# group_val = group_no
  list_of_files = cluster
  copy_path = dcom_folder+patient+'.nii.gz/'
  paste_path = g64+patient+'_'+str(group_val)
  print('\ncopying from',copy_path)
  print('saving',paste_path)
  if os.path.exists(paste_path) is not True:
    os.mkdir(paste_path)
  for i in list_of_files:
    shutil.copy(copy_path+i,paste_path+'/'+i)
  

def sequential_series(multiplier,path):
  h = []
  
  
  for i in range(multiplier*64, (multiplier+1)*64):
    h.append(path+'/slice'+str(i)+'.dcm')
  #   k.append('slice'+str(i)+'.dcm')
  # print(k)
  ranger = h[0].split('/')[-1]+' to '+h[-1].split('/')[-1]
  print('slice range',ranger,'\nsize:',len(h))
  return tuple(h)



In [ ]:
def prepare(in_dir, kfold_train, kfold_test, pixdim=(1.5, 1.5, 1.0), a_min=-200, a_max=200, spatial_size=[128,128,64], cache=False):

    """
    This function is for preprocessing, it contains only the basic transforms, but you can add more operations that you 
    find in the Monai documentation.
    https://monai.io/docs.html
    """

    set_determinism(seed=0)

    path_train_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))
    path_train_volumes = [path_train_volumes[x] for x in kfold_train]

    path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))
    path_train_segmentation = [path_train_segmentation[x] for x in kfold_train]

    path_test_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))
    path_test_volumes = [path_test_volumes[x] for x in kfold_test]

    path_test_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))
    path_test_segmentation = [path_test_segmentation[x] for x in kfold_test]

    
    train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
    test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]

    train_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

        ]
    )

    test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max,b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

            
        ]
    )

    if cache:
        train_ds = CacheDataset(data=train_files, transform=train_transforms,cache_rate=1.0)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader

    else:
        train_ds = Dataset(data=train_files, transform=train_transforms)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = Dataset(data=test_files, transform=test_transforms)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader


In [ ]:
from monai.utils import first
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
from monai.losses import DiceLoss
from tqdm import tqdm

def dice_metric(predicted, target):
    '''
    In this function we take `predicted` and `target` (label) to calculate the dice coeficient then we use it 
    to calculate a metric value for the training and the validation.
    '''
    dice_value = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
    value = 1 - dice_value(predicted, target).item()
    return value

def calculate_weights(val1, val2):
    '''
    In this function we take the number of the background and the forgroud pixels to return the `weights` 
    for the cross entropy loss values.
    '''
    count = np.array([val1, val2])
    summ = count.sum()
    weights = count/summ
    weights = 1/weights
    summ = weights.sum()
    weights = weights/summ
    return torch.tensor(weights, dtype=torch.float32)

def train(model, data_in, loss, optim, max_epochs, model_dir, test_interval=1 , device=torch.device("cuda:0")):
    best_metric = -1
    best_metric_epoch = -1
    try:
      save_loss_train = list(np.load(os.path.join(model_dir, 'loss_train.npy')))
      save_loss_test = list(np.load(os.path.join(model_dir, 'loss_test.npy')))
      save_metric_train = list(np.load(os.path.join(model_dir, 'metric_train.npy')))
      save_metric_test = list(np.load(os.path.join(model_dir, 'metric_test.npy')))
    except FileNotFoundError:
      save_loss_train = []
      save_loss_test = []
      save_metric_train = []
      save_metric_test = []
    train_loader, test_loader = data_in
    

    for epoch in range(max_epochs):
        # clear()
        # try:
        #   plt= 0
        #   epoch_checker()
        # except FileNotFoundError:
        #   print('Not doing the EPOCH CHECKER')
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        
        model.train()
        try:
          train_epoch_loss = save_loss_train[-1]
          epoch_metric_train = save_metric_train[-1]
        except IndexError:
          train_epoch_loss = 0
          epoch_metric_train = 0
          
        train_step = 0
        
        for batch_data in train_loader:
            clear()
            train_step += 1

            volume = batch_data["vol"]
            label = batch_data["seg"]
            label = label != 0
            volume, label = (volume.to(device), label.to(device))

            optim.zero_grad()
            outputs = model(volume)
            
            train_loss = loss(outputs, label)
            
            train_loss.backward()
            optim.step()

            train_epoch_loss += train_loss.item()
            print(
                f"{train_step}/{len(train_loader) // train_loader.batch_size}, "
                f"Train_loss: {train_loss.item():.4f}")

            train_metric = dice_metric(outputs, label)
            epoch_metric_train += train_metric
            print(f'Train_dice: {train_metric:.4f}')
            !nvidia-smi
            # time.sleep(1)
            # clear()

        clear()
        print('-'*20)
        
        train_epoch_loss /= train_step
        print(f'Epoch_loss: {train_epoch_loss:.4f}')
        save_loss_train.append(train_epoch_loss)
        np.save(os.path.join(model_dir, 'loss_train.npy'), save_loss_train)
        
        epoch_metric_train /= train_step
        print(f'Epoch_metric: {epoch_metric_train:.4f}')

        save_metric_train.append(epoch_metric_train)
        np.save(os.path.join(model_dir, 'metric_train.npy'), save_metric_train)
        
        
        if (epoch + 1) % test_interval == 0:

            model.eval()
            with torch.no_grad():
                try:
                  test_epoch_loss = save_loss_test[-1]
                  epoch_metric_test = save_metric_test[-1]
                except IndexError:
                  test_epoch_loss = 0
                  epoch_metric_test = 0

                test_metric = 0
                
                test_step = 0

                for test_data in test_loader:
                    # clear()
                    test_step += 1

                    test_volume = test_data["vol"]
                    test_label = test_data["seg"]
                    test_label = test_label != 0
                    test_volume, test_label = (test_volume.to(device), test_label.to(device),)
                    
                    test_outputs = model(test_volume)
                    
                    test_loss = loss(outputs, test_label)
                    test_epoch_loss += test_loss.item()
                    test_metric = dice_metric(test_outputs, test_label)
                    epoch_metric_test += test_metric


                    
                
                test_epoch_loss /= test_step
                print(f'test_loss_epoch: {test_epoch_loss:.4f}')
                
                # clear()
                save_loss_test.append(test_epoch_loss)
                np.save(os.path.join(model_dir, 'loss_test.npy'), save_loss_test)

                epoch_metric_test /= test_step
                print(f'test_dice_epoch: {epoch_metric_test:.4f}')
                save_metric_test.append(epoch_metric_test)
                np.save(os.path.join(model_dir, 'metric_test.npy'), save_metric_test)
                clear()
                print('checking epoch...')
                time.sleep(1)
                epoch_checker()
                clear()
                
                
                if epoch_metric_test > best_metric:
                    best_metric = epoch_metric_test
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), os.path.join(
                        model_dir, "best_metric_model.pth"))
                
                print(
                    f"current epoch: {epoch + 1} current mean dice: {test_metric:.4f}"
                    f"\nbest mean dice: {best_metric:.4f} "
                    f"at epoch: {best_metric_epoch}"
                  )
        


    print(
        f"train completed, best_metric: {best_metric:.4f} "
        f"at epoch: {best_metric_epoch}")
    
  

def calculate_pixels(data):
    val = np.zeros((1, 2))

    for batch in tqdm(data):
        batch_label = batch["seg"] != 0
        _, count = np.unique(batch_label, return_counts=True)

        if len(count) == 1:
            count = np.append(count, 0)
        val += count

    print('The last values:', val)
    return val

def timenow():
  from datetime import date as datetimedate, datetime, timedelta
  import pytz
  return datetime.now(pytz.timezone("Etc/GMT-6")).strftime('%d %B %Y, %I:%M:%S %p')
# timenow()
def epoch_checker():
  from matplotlib import pyplot as plt
  train_loss = np.load(os.path.join(model_dir, 'loss_train.npy'))
  train_metric = np.load(os.path.join(model_dir, 'metric_train.npy'))
  test_loss = np.load(os.path.join(model_dir, 'loss_test.npy'))
  test_metric = np.load(os.path.join(model_dir, 'metric_test.npy'))
  fig,ax = plt.subplots(figsize=(6,12),nrows=2,ncols=1,)
  
  a, = ax[0].plot(test_loss,'o-',label='test_loss')
  b, = ax[0].plot(train_loss,'x--',label='train_loss')
  # ax.grid()
  ax[0].set_title('DICE loss')
  ax[0].legend(handles=[a,b])
  # plt.legend()
  ax[0].grid()
  # plt.subplot(2,1,2)
  
  a, = ax[1].plot(test_metric,'o-',label='test_metric')
  b, = ax[1].plot(train_metric,'x--',label='train_metric')

  ax[1].legend(handles=[a,b])
  ax[1].set_title('DICE metric')
  plt.suptitle('\n\nepoch'+sep+str(len(train_loss))+'\n'+timenow())
  ax[1].grid()
  # if os.path.exists(model_dir+'graphs/') is False:
  #   os.mkdir(model_dir+'graphs/')
  # plt.legend()
  
  plt.savefig(model_dir+'graphs/'+timenow()+'.png',bbox_inches='tight')
  # plt = 0
  
# epoch_checker()


In [ ]:
# try:
# train(model, data_in, loss_function, optimizer, 20, model_dir)
# except ZeroDivisionError:
#   clear()
#   train(model, data_in, loss_function, optimizer, 600, model_dir)
def timenow():
  from datetime import date as datetimedate, datetime, timedelta
  import pytz
  return datetime.now(pytz.timezone("Etc/GMT-6")).strftime('%d %B %Y, %I:%M:%S %p')

In [ ]:
##############JUST PLOTTING
import numpy as np, nibabel
from matplotlib import pyplot as plt
from monai.transforms import(
  Compose,
  AddChanneld,
  LoadImaged,
  Resized,
  ToTensord,
  Spacingd,
  Orientationd,
  ScaleIntensityRanged,
  CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism
from monai.utils import first
from monai.utils import first, set_determinism
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Activations,
)

from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.data import CacheDataset, DataLoader, Dataset
from monai.inferers import sliding_window_inference

In [ ]:

trpath = main_path+'imagesTr_path'
labels = main_path+'labels_path'
tspath = main_path+'imagesTs_path'
traindir = main_path+'TRAINER/'
tsseg = traindir+'TestSegmentation/'
tsvol = traindir+'TestVolumes/'
trseg = traindir+'TrainSegmentation/'
trvol = traindir+'TrainVolumes/'
data_dir = traindir
model_dir = main_path+'MODELscratch/'
kfold = main_path+'KFold/'



In [ ]:
def predicting_liver_mask_for_test_files_without_segmentation(test_files,start_slice,end_slice,colormap='grey',model_path=main_path+'/MODELscratch/best_metric_model.pth'):
# colormap = 'gray'
  file_name = test_files[0]['vol'].split(tsvol)[-1]
  print('Validating against',test_files[0]['vol'].split(trvol)[-1])
  print('from slice',start_slice,'to',end_slice)


  test_transforms = Compose(
      [
          LoadImaged(keys=["vol", "seg"],allow_missing_keys=True),
          AddChanneld(keys=["vol", "seg"],allow_missing_keys=True),
          Spacingd(keys=["vol", "seg"],allow_missing_keys=True, pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
          Orientationd(keys=["vol", "seg"],allow_missing_keys=True, axcodes="RAS"),
          ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
          CropForegroundd(keys=['vol', 'seg'], source_key='vol',allow_missing_keys=True),
          Resized(keys=["vol", "seg"],allow_missing_keys=True, spatial_size=[128,128,64]),   
          ToTensord(keys=["vol", "seg"],allow_missing_keys=True),
      ]
  )

  test_ds = Dataset(data=test_files, transform=test_transforms)

  # test_ds = Dataset(data=test_files)
  test_loader = DataLoader(test_ds, batch_size=1)
  device = torch.device(my_device)
  model = UNet(
      dimensions=3,
      in_channels=1,
      out_channels=2,
      channels=(16, 32, 64, 128, 256), 
      strides=(2, 2, 2, 2),
      num_res_units=2,
      norm=Norm.BATCH,
  ).to(device)
  if model_path not in empty_list:
    model.load_state_dict(torch.load(
        model_path,map_location=torch.device(my_device)))
    model.eval()

  if model_path in empty_list:
    model_path = '**no models were used**'
  sw_batch_size = 4
  roi_size = (128, 128, 64)
  with torch.no_grad():
      test_patient = first(test_loader)
      t_volume = test_patient['vol']
      # t_segmentation = test_patient['seg']
      
      test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
      # tarp = test_outputs
      sigmoid_activation = Activations(sigmoid=True)
      test_outputs = sigmoid_activation(test_outputs)
      # test_outputs = test_outputs > 0.997
      test_outputs = test_outputs > .75
      # import cv2          
      for i in range(start_slice,end_slice+1):
          # plot the slice [:, :, 80]
          processed_slice = test_patient["vol"][0, 0, :, :, i].numpy()
          # target = test_patient["seg"][0, 0, :, :, i].numpy() > 0
          target= np.ones([128,128],dtype='float32')
          actual_slice = nibabel.load(tsvol+file_name).get_fdata()[:,:,i]
          prediction = np.array(test_outputs.detach().cpu()[0,1,:,:,i],dtype='float32')
          difference = np.array(np.logical_xor(processed_slice,prediction),dtype='float32')
          # difference = 1-(2*(np.logical_and(target,prediction)/np.logical_or(target,prediction)))

          processed_slice = correct_orient(processed_slice)
          target = correct_orient(target)
          actual_slice = correct_orient(actual_slice)
          prediction = correct_orient(prediction)
          difference = correct_orient(difference)

          try:
            accuracy = round((1-(abs(np.count_nonzero(difference))/np.count_nonzero(target)))*100,3)
            print('xor accuracy:',accuracy)
            # accuracy = round(dice_coef(prediction,target)*100,3)
            accuracy = round(np.sum(prediction[target==1]*2.0/(np.sum(prediction)+np.sum(target)))*100,3)
            print('dice coeff:',accuracy)
            
          except ZeroDivisionError:
            if np.count_nonzero(target)==0 and np.count_nonzero(prediction) != 0:
              accuracy = 0
            if np.count_nonzero(target)!=0  and np.count_nonzero(prediction) == 0:
              accuracy = 0
            if np.count_nonzero(target)==0 and np.count_nonzero(prediction) == 0:
              accuracy = 100
          
          
          # plt.rc('lines',linewidth=1.6)
          
          fig, ax = plt.subplots(figsize=(30,12))
          plt.rc('font',family='serif',size=35)
          COLOR = [0,0,0]
          # ax.tick_params(size=14)
          # ax.yticks(fontsize=14)
          plt.axis('on')
          plt.rcParams['axes.labelcolor'] = COLOR
          plt.rcParams['xtick.labelsize'] = 26
          plt.rcParams['ytick.labelsize'] = 0
          plt.rcParams['xtick.color'] = COLOR
          plt.rcParams['ytick.color'] = COLOR

          # plt.rcParams['text.color'] = [1,1,1]
          plt.subplot(1,5,1)
          plt.title(f"SLICE\n")
          plt.imshow(actual_slice,cmap=colormap)
          plt.xticks([0,actual_slice.shape[0]])
          plt.yticks([0,0])

          plt.subplot(1, 5, 2)
          plt.title(f"PROCESSED\nSLICE")
          plt.imshow(processed_slice,cmap=colormap)
          plt.xticks([0,processed_slice.shape[0]])
          plt.yticks([0, 0])
          
          plt.subplot(1, 5, 4)
          plt.title(f"OVERLAY\n")
          mask = prediction
          plt.imshow(processed_slice,cmap=colormap)
          plt.imshow(np.ma.masked_where(mask==0,mask),cmap='jet')
          
          
          plt.xticks([0,processed_slice.shape[0]])
          plt.yticks([0,0])
          
          plt.subplot(1, 5, 3)
          plt.title(f"PREDICTION\n")
          plt.imshow(prediction,cmap=colormap)
          plt.xticks([0,prediction.shape[0]])
          plt.yticks([0,0])
          
          # clear()
          plt.subplot(1,5,5)
          plt.title(f"SEGMENTED\nPART")
          plt.imshow(processed_slice, cmap='gray', alpha=1.0*(prediction>0))
          plt.xticks([0,processed_slice.shape[0]])
          plt.yticks([0,0])
          # plt.suptitle('TEST FILE: '+tsvol+file_name+'\nSLICE NUMBER: '+str(i)+'\n'+
          #             'MODEL USED: '+model_path.split(main_path)[-1]+'\n'+
          #             'DEVICE: '+my_device.upper()+'\n'+
          #             # 'THRESHOLD: '+str(threshold_of_error)+' [0 --> 1]\n'+
          #             # 'ACCURACY: '+str(accuracy)+'%\n'+
          #             'TIMESTAMP: '+timenow(),ha='left',va='top',x=0.11,y=.89)
          if accuracy > 10 and accuracy < 100:
            # clear()
            file_name = model_path.split(main_path)[-1].split('/')[2]+' [segmenting]'
            print(file_name)
            plt.savefig(main_path+'TESTED_DATA/'+file_name+sep+timenow()+'.png',bbox_inches='tight',dpi=600)
            plt.show()
            print('SAVED VALIDATION....')
          
          # plt.savefig(main_path+'TESTED_DATA/TESTVOLUME '+timenow()+'.png',bbox_inches='tight',dpi=600)
          # plt.show()
          # print('SAVED VALIDATION....')         

In [ ]:
def predicting_liver_mask_for_new_files(file_index,start_slice,end_slice,colormap='gray',threshold_of_error=0.53,model_path=main_path+'/MODELscratch/best_metric_model.pth'):

  
  my_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  print('CHOOSING',my_device,'to do the validation')

  in_dir = traindir
  
  path_train_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[0:index]
  path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[0:index]

  path_test_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[index:]
  path_test_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[index:]

  train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
  test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]
  # test_files = test_files
  # print(test_files)
  # test_files = [{'vol':tsvol+'liver_132_0.nii.gz','seg':trseg+'liver_0_0.nii.gz'}]

  test_files = test_files[file_index-1:file_index]
  file_name = test_files[0]['vol'].split(trvol)[-1]
  print('Validating against',test_files[0]['vol'].split(trvol)[-1])
  print('from slice',start_slice,'to',end_slice)


  test_transforms = Compose(
      [
          LoadImaged(keys=["vol", "seg"]),
          AddChanneld(keys=["vol", "seg"]),
          Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
          Orientationd(keys=["vol", "seg"], axcodes="RAS"),
          ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
          CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
          Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),   
          ToTensord(keys=["vol", "seg"]),
      ]
  )

  test_ds = Dataset(data=test_files, transform=test_transforms)

  # test_ds = Dataset(data=test_files)
  test_loader = DataLoader(test_ds, batch_size=1)
  device = torch.device(my_device)
  model = UNet(
      dimensions=3,
      in_channels=1,
      out_channels=2,
      channels=(16, 32, 64, 128, 256), 
      strides=(2, 2, 2, 2),
      num_res_units=2,
      norm=Norm.BATCH,
  ).to(device)
  if model_path not in empty_list:
    model.load_state_dict(torch.load(
        model_path,map_location=torch.device(my_device)))
    model.eval()

  if model_path in empty_list:
    model_path = '**no models were used**'
  sw_batch_size = 4
  roi_size = (128, 128, 64)
  with torch.no_grad():
      test_patient = first(test_loader)
      t_volume = test_patient['vol']
      t_segmentation = test_patient['seg']
      
      test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
      # tarp = test_outputs
      sigmoid_activation = Activations(sigmoid=True)
      test_outputs = sigmoid_activation(test_outputs)
      # test_outputs = test_outputs > 0.997
      test_outputs = test_outputs > .75
      # import cv2          
      for i in range(start_slice,end_slice+1):
          # plot the slice [:, :, 80]
          processed_slice = test_patient["vol"][0, 0, :, :, i].numpy()
          target = test_patient["seg"][0, 0, :, :, i].numpy() > 0
          actual_slice = nibabel.load(trvol+file_name).get_fdata()[:,:,i]
          prediction = np.array(test_outputs.detach().cpu()[0,1,:,:,i],dtype='float32')
          difference = np.array(np.logical_xor(prediction,target),dtype='float32')
          # difference = 1-(2*(np.logical_and(target,prediction)/np.logical_or(target,prediction)))

          
          processed_slice = correct_orient(processed_slice)
          target = correct_orient(target)
          actual_slice = correct_orient(actual_slice)
          prediction = correct_orient(prediction)
          difference = correct_orient(difference)

          try:
            accuracy = round((1-(abs(np.count_nonzero(difference))/np.count_nonzero(target)))*100,3)
            print('xor accuracy:',accuracy)
            # accuracy = round(dice_coef(prediction,target)*100,3)
            accuracy = round(np.sum(prediction[target==1]*2.0/(np.sum(prediction)+np.sum(target)))*100,3)
            print('dice coeff:',accuracy)
            
          except ZeroDivisionError:
            if np.count_nonzero(target)==0 and np.count_nonzero(prediction) != 0:
              accuracy = 0
            if np.count_nonzero(target)!=0  and np.count_nonzero(prediction) == 0:
              accuracy = 0
            if np.count_nonzero(target)==0 and np.count_nonzero(prediction) == 0:
              accuracy = 100
          
          
          # plt.rc('lines',linewidth=1.6)
          
          fig, ax = plt.subplots(figsize=(30,12))
          plt.rc('font',family='serif',size=35)
          COLOR = [0,0,0]
          # ax.tick_params(size=14)
          # ax.yticks(fontsize=14)
          plt.axis('on')
          plt.rcParams['axes.labelcolor'] = COLOR
          plt.rcParams['xtick.labelsize'] = 26
          plt.rcParams['ytick.labelsize'] = 0
          plt.rcParams['xtick.color'] = COLOR
          plt.rcParams['ytick.color'] = COLOR

          plt.subplot(1,5,1)
          plt.title(f"SLICE\n")
          plt.imshow(actual_slice,cmap=colormap)
          plt.xticks([0, actual_slice.shape[0]])
          plt.yticks([0, actual_slice.shape[0]])

          plt.subplot(1, 5, 2)
          plt.title(f"PROCESSED\nSLICE")
          plt.imshow(processed_slice,cmap=colormap)
          plt.xticks([0, processed_slice.shape[0]])
          plt.yticks([0, 0])
          
          plt.subplot(1, 5, 3)
          plt.title(f"TARGET\n")
          plt.imshow(target,cmap=colormap)
          plt.xticks([0, target.shape[0]])
          plt.yticks([0, 0])
          # plt.imshow(nibabel.load(test_files[0]['seg']).get_fdata()[:,:,i])
          plt.subplot(1, 5, 4)
          plt.title(f"PREDICTION\n")
          plt.imshow(prediction,cmap=colormap)
          plt.xticks([0, prediction.shape[0]])
          plt.yticks([0, 0])
          
          # clear()
          plt.subplot(1,5,5)
          plt.title(f"DIFFERENCE\n")
          plt.imshow(difference,cmap=colormap)
          plt.xticks([0, difference.shape[0]])
          plt.yticks([0, 0])
          # plt.suptitle('TEST FILE: '+file_name+'\nSLICE NUMBER: '+str(i)+'\n'+
          #             'MODEL USED: '+model_path.split(main_path)[-1]+'\n'+
          #             'DEVICE: '+my_device.upper()+'\n'+
          #             'THRESHOLD: '+str(threshold_of_error)+' [0 --> 1]\n'+
          #             'ACCURACY: '+str(accuracy)+'%\n'+
          #             'TIMESTAMP: '+timenow(),ha='left',va='top',x=0.11,y=.92)
          
          
          if accuracy > 88 and accuracy < 100:
            # clear()
            file_name = model_path.split(main_path)[-1].split('/')[2]
            print(file_name)
            plt.savefig(main_path+'TESTED_DATA/'+file_name+sep+timenow()+'.png',bbox_inches='tight',dpi=600)
            plt.show()
            print('SAVED VALIDATION....')

  # return accuracy

In [ ]:
# in_dir = traindir
# index = 331
# path_train_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[0:index]
# path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[0:index]

# path_test_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[index:]
# path_test_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[index:]
# path_test_volumes[0],path_train_volumes[-1]

In [ ]:
# a_min = 200
# a_max = -a_min
# size = 200
# patient = prepare(traindir,a_max=a_max,a_min=a_min,spatial_size=[size,size,64])
# show_patient(patient,63)

In [ ]:
##############JUST PLOTTING
def shows_processed_image_pipeline_for_a_random_image():
  import numpy as np, nibabel
  from matplotlib import pyplot as plt
  from monai.transforms import(
      Compose,
      AddChanneld,
      LoadImaged,
      Resized,
      ToTensord,
      Spacingd,
      Orientationd,
      ScaleIntensityRanged,
      CropForegroundd,
  )
  from monai.data import DataLoader, Dataset, CacheDataset
  from monai.utils import set_determinism
  from monai.utils import first
  in_dir = traindir
  main_path = '/content/drive/MyDrive/thesis_dataset/'
  livpath = main_path+'Liver/Task03_Liver/'
  mypath = main_path+'test_path/'
  trpath = main_path+'imagesTr_path'
  labels = main_path+'labels_path'
  tspath = main_path+'imagesTs_path'
  folder = 'imagesTr'
  sixtyfournifties = main_path+'groups_of_64_nifti/'+folder+'/'



  inc = 0
  index = np.random.randint(0,366)

  slice_number1 = np.random.randint(0,64)
  # print(index,file_index,slice_number1)


  path_test_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[index:]
  path_test_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[index:]
  test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]

  file_index = np.random.randint(0,len(test_files)-1)
  test_files = test_files[file_index-1:file_index]
  file_name1 = test_files[0]['vol']


  # file_name1

  slice_number = int(file_name1.split('.nii.gz')[0].split('_')[-1])*64+slice_number1


  # files = os.listdir(sixtyfournifties)
  test_image1 = nibabel.load(file_name1).get_fdata()
  # test_image2 = nibabel.load(file_name2).get_fdata()
  test_image1 = correct_orient(test_image1)
  # test_files = [{'vol':file_name1,'seg':file_name1.replace('Volumes','Segmentation')}]

  plt = ''
  ax = ''
  fig = ''
  from matplotlib import pyplot as plt
      
  fig,ax = plt.subplots(figsize=(23,14))
  
  plt.rc('font',family='serif',size=35)
  COLOR = [0,0,0]
  # ax.tick_params(size=14)
  # ax.yticks(fontsize=14)
  # plt.axis('on')
  plt.rcParams['axes.labelcolor'] = COLOR
  plt.rcParams['xtick.labelsize'] = 26
  plt.rcParams['ytick.labelsize'] = 0
  plt.rcParams['xtick.color'] = COLOR
  plt.rcParams['ytick.color'] = COLOR


  plt.subplot(2,4,1)
  
  str0 = '1. Loading\nSlice'
  plt.imshow(test_image1[:,:,slice_number1],cmap='gray')
  plt.xticks([0, test_image1.shape[0]])
  plt.yticks([0, test_image1.shape[0]-1])
  plt.title(str0)


  str1 = '2. Adding an\nextra channel'
  plt_number = 2
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            # Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            # Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            # ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
            # CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),   
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)

  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])


  str1 = '3. Bilinear\nSpacing'
  plt_number = 3
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            # Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            # ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
            # CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),   
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  str1 = '4. Correcting\nOrientation'
  plt_number = 4
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            # ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
            # CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),   
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  str1 = '5. Scaling\nIntensity'
  plt_number = 5
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
            # CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),   
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  str1 = '6. Croping\nForeground'
  plt_number = 6
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),   
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  a,b = test_image2.shape
  
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  # plt.xticks([0, test_image2.shape[1]])
  # plt.yticks([test_image2.shape[0]-1-test_image2.shape[1], test_image2.shape[0]-1])
  plt.xticks([0,a])
  plt.yticks([test_image2.shape[0]-1-b,test_image2.shape[0]-1])
  str1 = '7. Resizing\n'
  plt_number = 7
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),   
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  str1 = '8. Processed\nSlice'
  plt_number = 8
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),   
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  # plt.suptitle('Processing '+file_name1.split(trvol)[-1]+' at slice number '+str(slice_number1))
  plt.savefig(main_path+'TESTED_DATA/PREPROCESSING IMAGE '+timenow()+'.png',bbox_inches='tight',dpi=600)
  plt.show()
  print('SAVED PREP....')
  plt.show()


In [ ]:
# epoch_checker()
clear()
my_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('CHOOSING',my_device,'to do the training')

In [ ]:
# ##getting the file sizes of the directories in the thesis_database folder
# dontcheck = ['Liver','imagesTr_path','imagesTs_path','labelsTr','groups_of_64','groups_of_64_nifti','groups_of_64_nifti_non_empty']
# for files in os.listdir(main_path):
#   if files in dontcheck:
#     continue
#   try:
#     print(files,sep,format_size(get_dir_size(main_path+files)))
#   except NotADirectoryError:
#     getfilesize(main_path+files,'')

CTRL+START
#ORGAN SEGMENTATION (LIVER) OF 3D MEDICAL IMAGES USING PYTORCH & MONAI
###GROUP 4, Md. Salim Shahed Shajid [2017338016], MD. Ashraf Hossain Ifty [2017338042]

thesis_database folder: https://drive.google.com/drive/folders/1VBjfxsrTDK6jChNYg4QCVIurLydUP0fB

Add this folder as a shortcut in the root folder 'My Drive.'

#FOLDERS OF INTEREST
####TRAINER -- contains the data for training and testing.
####TESTED_DATA -- contains the validation of the models.

#MODEL folders
1. MODEL
2. MODELscratch
3. MODELscratch_2


#Summary

Dataset from the Decathlon medical image website [http://medicaldecathlon.com/] was collected. It is the Task03_Liver.tar in the folder. Data were extracted [thesis_database/Liver] and then because of uneven slices for each CT scan file [thesis_database/imagesTr_path], we sliced the standalone files in packets of 64 sliced nifti files [.nii.gz]. The segmented slices [thesis_database/labels_path] were also packeted in a similar fashion. 
After this, we discarded the nifti files that had no segmentation [the corresponding labels file had no data] and put them in the TRAINER folder. The actual slices were copied to the [thesis_database/TRAINER/TrainVolumes] folder, and the segmentation [labels] were copied to the [thesis_database/TRAINER/TrainSegmentation] folder. 

Then we took files from the TrainVolumes folder, and processed them further to make the training less time consuming. The processing part was to resize the image from [512,512] pixel to [128,128], the foreground [pixels with significant data] and background [pixels with not enough data] were saturated, the orientation was normalized in a consistent permutation i.e. RAS orientation. Then, the image's voxel width and height, were changed to 1.5 to make the processing faster and we further contrasted the images foreground to make the features more prominent. 

We split the TrainVolume data into random slices for Training and Testing the models accuracy. After that the data were cached into the gpu memory for faster training time, and now, we have a few trained models at various epochs. 

In the validation part of this system, the model takes in the processed image and predicts a mask where the liver should be. We compare this prediction with the actual label and determine how accurate the prediction was. The comparism is a simple XOR operation between the two images. So, when both pixel value is 1, the result is 0. This why we find the inaccuracy of the prediction and the target.



In [ ]:
def correct_orient(a):
  a = np.fliplr(a)
  a = np.rot90(a,3)
  return a

In [ ]:
###BEFORE DOING ANYTHING, CTRL+F8, TO RUN ABOVE CELLS AND THEN RUN THIS CELL 
###AND MOUNT DRIVE
###TO SEE IF EVERYTHING IS IN ORDER
###TRAINER FOLDER CANNOT BE 0 BYTES
dont_check = ['Liver','imagesTr_path','imagesTs_path',
              'labels_path','groups_of_64','groups_of_64_nifti','groups_of_64_nifti_non_empty','model_results']
for files in os.listdir(main_path):
  if files not in dont_check:
    try:
      print(files,sep,format_size(get_dir_size(main_path+files)))
    except NotADirectoryError:
      getfilesize(main_path+files,'')
    # print(files)


In [ ]:
fuse = main_path+'fusemachine kaggle/'
kag = main_path+'kaggler/'
kag = kag+'image-classification-computer-cv-course/'

train_folder = kag+'train.tar/train'
test_folder = kag+'test.tar/test'
val_folder = kag+'val.tar/val'

for item in os.listdir(kag):
  # if item in ['test.tar.xz']:
  if os.path.isdir(kag+item):
    print(item+sep+'[directory]')
    continue
  print(item)
#     shutil.unpack_archive(kag+item,kag+os.path.splitext(item)[0])

# # for item in os.listdir(kag):
# #   print(item)

In [ ]:
import pandas as pd

df = pd.read_csv(kag+'val.csv',header=None,names=['image_file','label'])

image_files = {}
i= 0
for item in df['image_file']:
  if df.iloc[i,0] in os.listdir(val_folder):
    image_files[i]={'location':val_folder+'/'+df.iloc[i,0],
                       'label':df.iloc[i,1]}
  # print(df.iloc[i,0])
  i+=1

# print(image_files)
file_name = val_folder+'/'+os.listdir(val_folder)[3]




In [ ]:
import random 

index = random.randint(0,len(image_files))
data = image_files[index]
image = data['location']
label = data['label']
print(image.split('/')[-1])
print('label:',label)
Image(image)


In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import layers

# # Load the data and labels from your dictionary
# data = []
# labels = []
# i = 0
# for index, item in image_files.items():
    
#     print(index)
#     display(image_files[index])
#     image = cv2.imread(item['location'])
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB format
#     image = cv2.resize(image, (224, 224)) # Resize to a consistent shape
#     data.append(image)
#     labels.append(item['label'])
#     clear()
#     i+=1


# Convert the data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=1000, validation_data=(test_data, test_labels))

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
from google.colab.patches import cv2_imshow

index = int(input())

image_path = image_files[index]['location']
label = image_files[index]['label']

import cv2


def crop_black_borders(image):
    # Convert the image to grayscale
    # image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply binary threshold to create a mask of the black regions
    _, mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    # Find contours of the black regions
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour
    max_contour = max(contours, key=cv2.contourArea)

    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(max_contour)

    # Crop the image to the bounding box
    cropped_image = image[y:y+h, x:x+w]

    # cropped_image = cv2.cvtColor(cropped_image,cv2.COLOR_BGR2GRAY)

    return cropped_image

def detect_tilted_rectangles(image_path):
    # Read the image and convert to grayscale
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to get a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours of all shapes
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours to get only rectangles with at least 4 corners
    rects = []
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4:
            rects.append(approx)

    # Count the number of dotted rectangles in the original image
    num_rects = 0
    for rect in rects:
        x, y, w, h = cv2.boundingRect(rect)
        roi = thresh[y:y+h, x:x+w]
        contours, _ = cv2.findContours(roi, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            if cv2.contourArea(cnt) < 10:
                continue
            rect = cv2.minAreaRect(cnt)
            if rect[1][0] > 50 and rect[1][1] > 2:
                num_rects += 1
    print('before rotating num_rects:',num_rects)
    # Rotate the image by 1 degree at a time until we find at least 20 rectangles
    angle = 0
    while num_rects < 20:
        # Rotate the image by 1 degree
        print('ANGLE:',angle,'num_rects:',num_rects)
        angle += 1
        rotated = imutils.rotate_bound(gray, angle)

        # Apply adaptive thresholding to get a binary image
        thresh = cv2.adaptiveThreshold(rotated, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

        # Find contours of all shapes
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours to get only rectangles with at least 4 corners
        rects = []
        for cnt in contours:
            approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
            if len(approx) == 4:
                rects.append(approx)

        # Count the number of dotted rectangles
        num_rects = 0
        for rect in rects:
            x, y, w, h = cv2.boundingRect(rect)
            roi = thresh[y:y+h, x:x+w]
            contours, _ = cv2.findContours(roi, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                if cv2.contourArea(cnt) < 10:
                    continue
                rect = cv2.minAreaRect(cnt)
                if rect[1][0] > 50 and rect[1][1] > 2:
                    num_rects += 1

    # Rotate the original image by the found angle and return it
    rotated = imutils.rotate_bound(img, angle)
    return rotated

import cv2

def crop_to_mini_rectangles(img):
    # Read the image and convert to grayscale
    # img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to get a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours of all shapes
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours to get only rectangles with at least 4 corners
    rects = []
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4:
            rects.append(approx)

    # Find the minimum and maximum x and y coordinates of all rectangles
    x_min = y_min = float('inf')
    x_max = y_max = float('-inf')
    for rect in rects:
        x, y, w, h = cv2.boundingRect(rect)
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x + w)
        y_max = max(y_max, y + h)

    # Crop the image to the area encompassed by the rectangles
    cropped = img[y_min:y_max, x_min:x_max]

    return cropped

import cv2
import numpy as np

def extract_rectangles(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to get a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours of all shapes
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours to get only rectangles with at least 4 corners
    rects = []
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4:
            rects.append(approx)

    # Sort the rectangles from left to right and top to bottom
    rects = sorted(rects, key=lambda x: (cv2.boundingRect(x)[1], cv2.boundingRect(x)[0]))

    # Extract the contents within each rectangle and add it to the array of images
    images = []
    for rect in rects:
        x, y, w, h = cv2.boundingRect(rect)
        roi = img[y:y+h, x:x+w]
        images.append(roi)

    return np.array(images)





import cv2
import numpy as np


rotated = detect_tilted_rectangles(image_path)
cropped = crop_black_borders(rotated)
mini_rect = extract_rectangles(image_path)


# gray = make_nonwhite_black(cropped)





fig, ax = plt.subplots(figsize=(18,6),facecolor=(1,1,1))
plt.suptitle('index: '+str(index)+'\n'+image_path.split('/')[-1]+'\nlabel: '+str(label))
plt.subplot(1,4,1)
plt.imshow(cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB))
plt.subplot(1,4,2)
plt.imshow(cv2.cvtColor(rotated,cv2.COLOR_BGR2RGB))
plt.subplot(1,4,3)
plt.imshow(cv2.cvtColor(cropped,cv2.COLOR_BGR2RGB))
plt.subplot(1,4,4)
plt.imshow(cv2.cvtColor(mini_rect,cv2.COLOR_BGR2RGB))

In [ ]:
import matplotlib.pyplot as plt

def show_images(images, nrows, ncols):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 10))
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(cv2.cvtColor(images[i],cv2.COLOR_BGR2RGB))
        ax.axis('off')
    plt.show()

# Assuming images is a list of 47 images
show_images(mini_rect, nrows=6, ncols=8)


In [ ]:
cv2_imshow(image[y:y+h,x:x+w])

In [ ]:
image

In [ ]:
for index, (image_files, label) in image_files.items():
  print(index,loc,label)

In [ ]:
##############SPLTTING OF THE TRAIN AND TEST FILES FOR KFOLD 
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=5)
# for kfold_train, kfold_test in kf.split(train_files):
#   # print('Train')
#   c+=1
#   actual_train_files = [train_files[x] for x in kfold_train]
#   # print(actual_train_files)
#   # print('length:',len(kfold_train))
#   # print('Test')
#   actual_test_files = [train_files[x] for x in kfold_test]
#   kfold_database['FOLD_'+str(c)]={'kfold_train':str(list(kfold_train)).strip('[]'),
#                                   'kfold_test':str(list(kfold_test)).strip('[]'),
#                                   'NUMBER OF TRAIN FILES':len(actual_train_files),
#                                  'TRAIN FILES':actual_train_files,
                                  
#                                  'NUMBER OF TEST FILES': len(actual_test_files),
#                                  'TEST FILES':actual_test_files}

#   # print(actual_test_files)
#   # print('length:',len(kfold_test))

# to_json(kfold_database,'kfold_database','write')
  # print('------------------------------------------------------')

In [ ]:
# #STARTING POINT,  
# #RUN THIS CELL TO LOAD ALL THE DATA IN THE MEMORY FOR FASTER TRAINING
# #YOU WILL GET RUNTIMEERROR THE FIRST TIME, JUST RE-RUN THIS CELL
# #AND AFTER THE DATA IS LOADED ONTO THE MEMORY RUN THE NEXT CELL TO TRAIN THE MODEL

# from monai.networks.nets import UNet
# import sys

# from monai.networks.layers import Norm
# from monai.losses import DiceLoss, DiceCELoss
# import torch

# #########################NOTEBOOKS IN PARALLEL OPERATION#############ONLY CHANGE THESE VARIABLES CLONE NUMBER FROM 1 TO 5, AND DESIRED EPOCHS
# CLONE_NUMBER = 1
# epochs = 100
# ########################################################


# model_dir = kfold+'DATA_FOLD_'+str(CLONE_NUMBER)+'/'

# print(model_dir)

# size = 128
# data_caches_for_different_kfold_splits = []

# train_files = sorted(glob(os.path.join(data_dir,"TrainVolumes","*.nii.gz")))
# # train_files = sorted(os.listdir(data_dir+'TrainVolumes'))
# train_files_segmentation = sorted(glob(os.path.join(data_dir,"TrainSegmentation","*.nii.gz")))

# kfold_database = to_json({},'kfold_database','read')['FOLD_'+str(CLONE_NUMBER)]

# kfold_train = [int(x) for x in kfold_database['kfold_train'].split(', ')]

# kfold_test = [int(x) for x in kfold_database['kfold_test'].split(', ')]

# if os.path.exists(model_dir+'graphs/') is False:
#   os.mkdir(model_dir+'graphs/')

# try:
#   epoch_done = len(np.load(model_dir+'loss_train.npy'))
# except Exception as e:
#   epoch_done = 0


# try:
#   print(data_in)
  
# except NameError:
    
  
#   if epochs-epoch_done >0:
#     data_in = prepare(data_dir, kfold_train, kfold_test,cache=True,spatial_size=[size,size,64])  #### THIS ONE TAKES DATA IN THE CACHE, TO MAKE THE TRAINING FAST
#     clone_k = CLONE_NUMBER

# if clone_k != CLONE_NUMBER and epochs-epoch_done>0:
#   data_in = prepare(data_dir, kfold_train, kfold_test,cache=True,spatial_size=[size,size,64])  #### THIS ONE TAKES DATA IN THE CACHE, TO MAKE THE TRAINING FAST
#   clone_k = CLONE_NUMBER

# print(model_dir)

In [ ]:
# #ACTUAL TRAINER, DOESNT MATTER IF TRAINING GETS INTERRUPTED, IT WILL DO THE TRAIN FROM THE
# #NEXT EPOCH, AS LONG AS NOTHING IS TOUCHED HERE
# device = torch.device(my_device)
# learn_rate =  1e-5

# model = UNet(
#     spatial_dims=3,
#     in_channels=1,
#     out_channels=2,
#     channels=(16, 32, 64, 128, 256),
#     strides=(2, 2, 2, 2),
#     num_res_units=2,
#     norm=Norm.BATCH,
# ).to(device)
# go = ''
# if os.path.exists(model_dir+'best_metric_model.pth'):
#   print('RESUMING FROM MODEL'+sep+model_dir.split('/')[-2])
#   model.load_state_dict(torch.load(
#         os.path.join(model_dir, "best_metric_model.pth"),map_location=torch.device(my_device)))#### to resume for a specific model
  
#   print(epoch_done,'epochs done.......')
#   if epochs-epoch_done > 0:
#     print('Remaining epochs',epochs-epoch_done)
    
#     # model.eval()
#     go='1'
# else:
#   go = '1'
#   epoch_done = 0
  
# # loss_function = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(3.32088658e+08, 2.54757580e+07).to(device))
# loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
# optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)

# if go not in empty_list:
#   print('STARTING TRAINING IN 5 SECONDS....')
#   time.sleep(5)
#   train(model, data_in, loss_function, optimizer, epochs-epoch_done, model_dir)
# # except ZeroDivisionError:
# #   clear()
# #   train(model, data_in, loss_function, optimizer, 600, model_dir)

In [ ]:
# ####DELETE (FAULTY) EPOCH INDEX 
# CL_NUMBER = 5
# fault_in_the_last = 0
# delete = 0 ################################### MAKE SURE THIS IS ALWAYS 0


# model_dir = kfold+'/DATA_FOLD_'+str(CL_NUMBER)+'/'

# if delete == 85695436:
#   delete_confirm = 1
#   files = ['loss_train','metric_train','loss_test','metric_test']
# else:
#   files = ['loss_train']
#   delete_confirm = ''


# for f in files:
#   if fault_in_the_last == 0:
    
#     a = np.load(model_dir+f+'.npy')
#   else:
#     a = np.load(model_dir+f+'.npy')[0:-fault_in_the_last]
  

#   if delete_confirm == 1:
#     print('reforming',model_dir+f+'.npy')
#     np.save(model_dir+f+'.npy',arr=a)
    
  
#   print(a,'\n')

#   print(len(a))
#   print('-------------------')



In [ ]:
################## HIGHLIGHTING VALUES OF INTEREST FROM THE KFOLDED MODELS

# from matplotlib import pyplot as plt

# models = [kfold+item+'/' for item in os.listdir(kfold)]

# data_points = ['loss_train','metric_train','loss_test','metric_test']

# c=0
# kfold_findings = to_json({},'kfold_findings','read')
# for model in models:
#   for item in data_points:
#     a = np.load(model+item+'.npy') 
#     e = len(a)
#     maxa, enda, mina, starta,mean,median = max(a),a[-1],min(a),a[0],np.mean(a),np.median(a)
#     if model.split('/')[-2] not in kfold_findings:
#       kfold_findings[model.split('/')[-2]]={}
    
#     kfold_findings[model.split('/')[-2]][item]={'max':maxa,
#                                                     'min':mina,
#                                                     'start':starta,
#                                                     'end':enda,
#                                                     'mean':mean,
#                                                     'median':median,
#                                                     'epochs':e}

# to_json(kfold_findings,'kfold_findings','write')



In [ ]:
# ######################## to make KFOLDED GRAPHS
# plt = ''
# ax = ''
# from matplotlib import pyplot as plt
# from matplotlib.ticker import FormatStrFormatter

# kfold_findings = to_json({},'kfold_findings','read')
# models = ['Model '+str(i+1) for i in range(len(kfold_findings))]
# max_val = ['metric_train','metric_test']
# min_val = ['loss_train','loss_test']

# COLOR = [0.149,0.149,0.149]
# plt.rc('font',family='serif',size=20)
# plt.rc('lines',linewidth=1.6)
# plt.rcParams['axes.labelcolor'] = COLOR
# plt.rcParams['xtick.labelsize'] = 16
# plt.rcParams['ytick.labelsize'] = 16

# plt.rcParams['xtick.color'] = COLOR
# plt.rcParams['ytick.color'] = COLOR


# fig, ax = plt.subplots(figsize=(10,8),facecolor=[1,1,1])
# ax.set_facecolor([1,1,1])
# ax.grid(visible=True,linewidth=0.20)

# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)
# ax.spines['bottom'].set_color(COLOR)
# ax.spines['bottom'].set_linewidth(1.6)
# ax.spines['left'].set_linewidth(1.6)

# x = []
# y = []
# c = 0
# for model in kfold_findings:
#   for item in max_val:###############################################################
#     if item.split('_')[0] == 'loss':
#       x.append(kfold_findings[model][item]['min'])
#     else:
#       x.append(kfold_findings[model][item]['max'])
#     y.append(c+1)
#   c+=1
# suf = item.split('_')[0].capitalize()

# if suf == 'Loss':
#   ax.set_ylim(0,.5)
# else:
#   suf = 'Score'
#   ax.set_ylim(.50,1)


# ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

# avg1,avg2 = ' [Average: '+str(round(np.mean(x[0::2]),2))+"]",' [Average: '+str(round(np.mean(x[1::2]),2))+"]"
# if suf != 'Loss':
    
#   COLOR = [255/255,10/255,10/255] ################ RED
#   ax.plot(models,x[0::2],'--x',linewidth=2.5,color = COLOR,label='Train Dice '+suf+avg1,markersize=16)
#   plt.bar(models,x[0::2],color=[255/255,200/255,200/255])

#   COLOR = [100/255,100/255,255/255]
#   ax.plot(models,x[1::2],'--o',linewidth=2.5,color = COLOR,label='Test Dice '+suf+avg2,markersize=8)
#   plt.bar(models,x[1::2],color=[200/255,200/255,255/255])



# else:

#   COLOR = [100/255,100/255,255/255]
#   ax.plot(models,x[1::2],'--o',linewidth=2.5,color = COLOR,label='Test Dice '+suf+avg2,markersize=8)
#   plt.bar(models,x[1::2],color=[200/255,200/255,255/255])

  
#   COLOR = [255/255,10/255,10/255] ################ RED
#   ax.plot(models,x[0::2],'--x',linewidth=2.5,color = COLOR,label='Train Dice '+suf+avg1,markersize=16)
#   plt.bar(models,x[0::2],color=[255/255,200/255,200/255])

  
  
# if suf != "Loss":
#   plt.ylabel('Dice Score\n')
# else:
#   plt.ylabel("Dice Loss\n")
# plt.xlabel('\nK-Folds')
# plt.legend(loc = 'upper right', frameon =False,bbox_to_anchor = (1.1,1.2))


# fig.savefig(main_path+'PAPER DATA/'+'KFOLD GRAPH'+sep+timenow()+'.png',bbox_inches='tight',facecolor=[1,1,1],dpi=600)

In [ ]:
##############JUST PLOTTING
import numpy as np, nibabel
from matplotlib import pyplot as plt
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism
from monai.utils import first

main_path = '/content/drive/MyDrive/thesis_dataset/'
livpath = main_path+'Liver/Task03_Liver/'
mypath = main_path+'test_path/'
trpath = main_path+'imagesTr_path'
labels = main_path+'labels_path'
tspath = main_path+'imagesTs_path'
folder = 'imagesTr'
sixtyfournifties = main_path+'groups_of_64_nifti/'+folder+'/'



inc = 0

for jk in range(0,1):

  file_num = np.random.randint(0,130)
  slice_number1 = np.random.randint(0,64)

  while True:
    try:
      sub_file = np.random.randint(0,10)
      file_name1 = sixtyfournifties+'liver_'+str(file_num)+'_'+str(sub_file)+'.nii.gz'
      file_name2 = livpath+'labelsTr/'+'liver_'+file_name1.split('.nii.gz')[0].split('_')[-2]+'.nii.gz'
      file_name3 = livpath+'imagesTr/'+'liver_'+file_name1.split('.nii.gz')[0].split('_')[-2]+'.nii.gz'
      slice_number = int(file_name1.split('.nii.gz')[0].split('_')[-1])*64+slice_number1
  # plt.rcParams['text.color'] = [1,1,1]

      files = os.listdir(sixtyfournifties)
      test_image1 = nibabel.load(file_name1).get_fdata()
      test_image2 = nibabel.load(file_name2).get_fdata()==1
      test_image3 = nibabel.load(file_name3).get_fdata()
    except:
      continue
    else:
      break

  ##################TO MAKE IT HOW IT LOOKS IN THE 3D SLICER
  test_image1 = correct_orient(test_image1)
  test_image2 = correct_orient(test_image2)
  test_image3 = correct_orient(test_image3)
  # test_image1 = np.fliplr(test_image1)
  # test_image1 = np.rot90(test_image1,3)
  # test_image2 = np.fliplr(test_image2)
  # test_image2 = np.rot90(test_image2,3)
  # test_image3 = np.fliplr(test_image3)
  # test_image3 = np.rot90(test_image3,3)
  #####################################################
  plt = ''
  ax = ''
  from matplotlib import pyplot as plt
  
  fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,5))
  plt.rc('font',family='serif',size=20)
  # plt.axis('off')
  COLOR = [1,1,1]
  plt.rcParams['axes.labelcolor'] = COLOR
  plt.rcParams['xtick.color'] = COLOR
  plt.rcParams['ytick.color'] = COLOR
  
  
  try:
    ax1.imshow(test_image1[:,:,slice_number1],cmap='gray')
    # ax1.set_title('SLICED\n'+file_name1.split('/')[-1])
    # ax1.set_title('SLICED ['+str(slice_number1)+'/'+str(test_image1.shape[-1]-1)+']\n'+file_name1.split('/')[-1])
    ax1.set_title('64 Sliced NIfTI\nImage')
  except IndexError:
    ax1.set_title('out of bounds')
  # ax2.set_title('ORIGINAL IMAGE ['+str(slice_number)+'/'+str(test_image3.shape[-1]-1)+']\n'+file_name3.split(livpath)[-1])
  ax2.set_title('64 Sliced NIfTI\nImage Segmentation')
  # plt.suptitle('Slice number '+str(slice_number))
  ax2.imshow(test_image2[:,:,slice_number],cmap='gray')
  # ax3.set_title('ORIGINAL LABEL ['+str(slice_number)+'/'+str(test_image2.shape[-1]-1)+']\n'+file_name2.split(livpath)[-1])
  # plt.suptitle('Slice number '+str(slice_number))
  # ax3.imshow(test_image2[:,:,slice_number],cmap='gray')
  plt.savefig(main_path+'TESTED_DATA/SLICING '+timenow()+'.png',bbox_inches='tight',dpi=600)
  plt.show()
  print('SAVED slice....')
  # plt.show()

In [ ]:
from matplotlib import pyplot as plt
model_dir = main_path+'model_results/with_dropout=0.5/'
model_dir = main_path+'model_results/without_dropout/'
# model_dir = main_path+'MODELscratch_2/'
# model_dir = main_path+'MODEL/'
try:
  extra = ''
  train_loss = np.load(os.path.join(model_dir, extra+ 'loss_train.npy'))
  train_metric = np.load(os.path.join(model_dir, extra+ 'metric_train.npy'))
  test_loss = np.load(os.path.join(model_dir, extra+ 'loss_test.npy'))
  test_metric = np.load(os.path.join(model_dir, extra+ 'metric_test.npy'))
except Exception as e:
  extra = 'Copy of '
  train_loss = np.load(os.path.join(model_dir, extra+ 'loss_train.npy'))
  train_metric = np.load(os.path.join(model_dir, extra+ 'metric_train.npy'))
  test_loss = np.load(os.path.join(model_dir, extra+ 'loss_test.npy'))
  test_metric = np.load(os.path.join(model_dir, extra+ 'metric_test.npy'))


plt.rc('font',family='serif',size=20)
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(12,6))




# plt.plot(test_loss,range(len(test_loss)))

# a, = ax[0].plot(test_loss,'o-',label='test_loss')
# b, = ax[0].plot(train_loss,'x--',label='train_loss')

# ax[0].set_title('DICE loss')
# ax[0].legend(handles=[a,b])
# # plt.legend()
# ax[0].grid()
# ax[0].set_yticks(np.linspace(min(test_loss),max(test_loss),10))
# ax[0].set_xticks(np.linspace(0,len(test_loss),10))
# plt.subplot(2,1,2)

##############METRIC
# suff = '_metric'
# a, = ax.plot(test_metric,'o-',label='Test Dice Score')
# b, = ax.plot(train_metric,'x--',label='Train Dice Score')

# ################LOSS
suff = '_loss'
a, = ax.plot(test_loss,'o-',label='Test Dice Loss Score')
b, = ax.plot(train_loss,'x--',label='Train Dice Loss Score')


# plt.rcParams['text.color'] = [0,0,0]
ax.legend(handles=[a,b])
# ax.set_title('DICE metric')
plt.xlabel('Epoch')
plt.ylabel('Dice Loss')
# plt.suptitle('Epoch'+sep+str(len(train_loss)),va='top',fontsize=22)
ax.grid()
# if os.path.exists(model_dir+'graphs/') is False:
#   os.mkdir(model_dir+'graphs/')
# plt.legend()
file_name = model_dir.split('/')[-2]+suff+'.png'
print('MODEL NAME:','_'.join(file_name.split('_')[0:2])+'\n')
print('Data points:',len(train_metric),'\n')
print('-------------METRICS----------------')
print('MAX TRAIN METRIC:',round(max(train_metric),3),', MAX TEST METRIC:',round(max(test_metric),3))
print('MIN TRAIN METRIC:',round(min(train_metric),3),', MIN TEST METRIC:',round(min(test_metric),3))
print('LAST TRAIN METRIC:',round((train_metric)[-1],3),', LAST TEST METRIC:',round((test_metric)[-1],3))

print('\n-------------LOSS METRICS----------------')
print('MAX TRAIN LOSS METRIC:',round(max(train_loss),3),', MAX TEST LOSS METRIC:',round(max(test_loss),3))
print('MIN TRAIN LOSS METRIC:',round(min(train_loss),3),', MIN TEST LOSS METRIC:',round(min(test_loss),3))
print('LAST TRAIN METRIC:',round((train_loss)[-1],3),', LAST TEST METRIC:',round((test_loss)[-1],3))
print('\n\n\n\n')
print(file_name)
plt.savefig(main_path+'TESTED_DATA/'+file_name,bbox_inches='tight',dpi=600)



In [ ]:
#this is the starting cell, ENTER CTRL+F8, TO RUN THE CELLS ABOVE
# to test random slices and getting predictions
# index = 331 
in_dir = traindir
for i in range(0,10):
  # clear()
  try:
    index = np.random.randint(0,366)

    slice_number1 = np.random.randint(0,64)
    # print(index,file_index,slice_number1)


    path_test_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[index:]
    path_test_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[index:]
    test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]
    
    ###########################################################################################
    file_index = np.random.randint(0,len(test_files)-1)
    start_slice, threshold_of_error = np.random.randint(0,64),np.random.randint(53,85)
    ############################################################################################


    test_files = test_files[file_index-1:file_index]
    file_name1 = test_files[0]['vol']
    
    
    end_slice = start_slice
    colormap = 'gray' #Grey, gray
    
    ################models
    # model_path = main_path+'/MODELscratch_2/best_metric_model.pth'
    model_path = main_path+'/model_results/without_dropout/best_metric_model.pth'
    # model_path = main_path+'/model_results/with_dropout=0.5/best_metric_model.pth'
    # model_path = main_path+'/MODELscratch/best_metric_model.pth'
    # model_path = ''        ###TO SEE WHAT HAPPENS WHEN NO MODEL IS USED

    # model_path = main_path+'/MODELscratch_2/best_metric_model.pth'
    # model_path = main_path+'/MODEL/best_metric_model.pth'
    # model_path = main_path+'/MODEL/best_metric_model_00.pth'
    # model_path = main_path+'/MODEL/best_metric_model_nogood.pth'
    

    ####################################


    predicting_liver_mask_for_new_files(file_index,
                                        start_slice,end_slice,
                                        colormap,
                                        threshold_of_error/100,
                                        model_path=model_path)
  except Exception as e:
    print(e)
    # clear()
  # continue

In [ ]:
# to test random slices and getting predictions TEST SLICES
# index = 331 
in_dir = traindir
for i in range(0,10):
  # clear()
  try:
    # index = np.random.randint(0,366)

    
    # print(index,file_index,slice_number1)


    path_test_volumes = sorted(glob(os.path.join(in_dir, "TestVolumes", "*.nii.gz")))
    test_files = [{"vol": image_name} for image_name in path_test_volumes]
    
    ###############################################################
    file_index = np.random.randint(0,len(test_files)-1)
    start_slice = np.random.randint(0,64)
    ###############################################################

    test_files = test_files[file_index-1:file_index]
    file_name1 = test_files[0]['vol']
    # print('file_name',file_name1)
    
    end_slice = start_slice
    colormap = 'gray' #Grey, gray
    
    ################models
    model_path = main_path+'/MODELscratch_2/best_metric_model.pth'
    model_path = main_path+'/MODELscratch/best_metric_model.pth'
    # model_path = ''        ###TO SEE WHAT HAPPENS WHEN NO MODEL IS USED

    # model_path = main_path+'/MODELscratch_2/best_metric_model.pth'
    # model_path = main_path+'/MODEL/best_metric_model.pth'
    # model_path = main_path+'/MODEL/best_metric_model_00.pth'
    # model_path = main_path+'/MODEL/best_metric_model_nogood.pth'
    model_path = main_path+'/model_results/without_dropout/best_metric_model.pth'

    ####################################
    predicting_liver_mask_for_test_files_without_segmentation(test_files,
                                        start_slice,end_slice,
                                        colormap,
                                        model_path=model_path)
  except Exception as e:
    print(e)
    # clear()
  # continue

In [ ]:
#selfexplanatory
for i in range(10):
  shows_processed_image_pipeline_for_a_random_image()

In [ ]:
###MAKES PDF 
from PIL import Image
pdpath = main_path+'PAPER DATA/'

if os.path.exists(pdpath+'/PDF/') is not True:
  print('making pdf folder')
  os.mkdir(pdpath+'/PDF/')

for item in os.listdir(pdpath):
  image_file = pdpath+item
  
  
  try:
    pdf_file_name = pdpath+'/PDF/'+os.path.splitext(item)[0]+'.pdf'
    if os.path.exists(pdf_file_name):
      getfilesize(pdf_file_name,'')
      continue
    else:
      im = Image.open(image_file).convert('RGB')
      im.save(pdf_file_name,dpi=(600,600))
      getfilesize(pdf_file_name,'')
    
  except IsADirectoryError:
    continue


In [ ]:
# for files in os.listdir(pdpath):
#   # im = Image.open(pdpath+files)
#   # print(files,sep,im.info['dpi'])
#   try:
#     os.remove(pdpath+files)
#   except IsADirectoryError:
#     continue

In [ ]:
################DICOM TO NIFTY CONVERTER USING SITK
# # labels = 74
# # trpath = 74
# # tspath = 42



# group_folder = 'labelsTr'

# copy_path = main_path+'groups_of_64/'+group_folder+'/'
# paste_path = main_path+'groups_of_64_nifti/'+group_folder+'/'

# file_name = 'liver_1_0'
# count = 0
# upto = len(os.listdir(copy_path))
# clone = 0
# a,b = (clone*int(np.ceil(upto/5)),(clone+1)*int(np.ceil(upto/5)))
# # os.listdir(copy_path)[a:b]
# upto = int(np.ceil(upto/5))
# # for file_name in (os.listdir(copy_path))[a:b]:

# mult = int(file_name.split('_')[-1])
# print('('+str(count+1)+'/'+str(upto)+')')
# if os.path.exists(paste_path+file_name+'nii.gz') is not True:
#   # break
#   # clear()
#   print('\nConverting',file_name)
#   print('dicom_series:',copy_path+file_name)
#   print('nifti_file:',paste_path+file_name+'.nii.gz')
  
#   # clear()
#   reader = sitk.ImageSeriesReader()
#   dicom_names = sequential_series(mult,copy_path+file_name)
#   reader.SetFileNames(dicom_names)
#   image = reader.Execute()
#   image = sitk.Flip(image,[True,False,False])
#   image = sitk.Flip(image,[False,True,False])


#   sitk.WriteImage(image, paste_path+file_name+'.nii.gz')
#   print('Conversion done')
#   clear()
# else:
#   j = sequential_series(mult,copy_path+file_name)
#   print(file_name,'already exists')
#   clear()

# count+=1


In [ ]:
#####NON EMPTY SEPARATOR
# group_folder = 'labelsTr'
# print('DOING',group_folder)
# g64n = main_path+'groups_of_64_nifti/'+group_folder+'/'
# g64ne = main_path+'groups_of_64_nifti_non_empty/'+group_folder+'/'
# number_of_files = len(os.listdir(g64n)),len(os.listdir(g64))

# upto = len(os.listdir(g64n))
# clone = 0
# a,b = (clone*int(np.ceil(upto/5)),(clone+1)*int(np.ceil(upto/5)))

# for files in tqdm(os.listdir(g64n)[a:b]):
#   # getfilesize(g64n+files,'')
#   f = nibabel.load(g64n+files).get_fdata()
#   if len(np.unique(f)) == 1:
#     print('\n',files)
#   else:
#     shutil.copy(g64n+files,g64ne+files)
#     shutil.copy(main_path+'groups_of_64_nifti/imagesTr/'+files,main_path+'groups_of_64_nifti_non_empty/imagesTr/'+files)


In [ ]:
#GETTING THE 64 SLICES DIRECTLY FROM NIFTY
# group_folder = 'imagesTr'

# og_path = livpath+group_folder+'/'
# upto = len(os.listdir(og_path))
# clone = 1
# c,d = (clone*int(np.ceil(upto/5)),(clone+1)*int(np.ceil(upto/5)))
# ext = '.nii.gz'
# count = 0

# # the_file_name = 'liver_0.nii.gz'

# for the_file_name in os.listdir(og_path)[c:d]:
#   if the_file_name.startswith('.') is not True:
#     copy_path = og_path+the_file_name
#     paste_path = main_path+'groups_of_64_nifti/'+group_folder+'/'+the_file_name.split(ext)[0]

#     original_nifty_file = load(copy_path)
#     number_of_slices = original_nifty_file.shape[-1]
#     clear()

#     tick = int(number_of_slices/64)
#     clear()
#     print('('+str(count+1)+'/'+str(upto)+')')
#     print('Converting',the_file_name)
#     print('partition number:',tick)
#     for i in range(tick):
#       if os.path.exists(paste_path+'_'+str(i)+ext) is not True:
#         a,b = i*64,(i+1)*64
#         roi  = np.asarray(original_nifty_file.dataobj[:,:,slice(a,b)])
#         img = Nifti1Image(roi,affine=original_nifty_file.affine)
#         # save()
#         print('\nSlice',a,'to',b,'from',copy_path.split(livpath)[-1])
#         print('Saving to',the_file_name.split(ext)[0]+'_'+str(i)+ext)
#         save(img,paste_path+'_'+str(i)+ext)
#       else:
#         clear()
#         print(the_file_name.split(ext)[0]+'_'+str(i)+ext,'is already saved.')

#     count+=1



In [ ]:
# print(traindir)
# for item in os.listdir(traindir):
#   print(item,sep,format_size(get_dir_size(traindir+item)))
#   print('number of items',sep,len(os.listdir(traindir+item)),'\n')

# print('\n',main_path+'groups_of_64_nifti_non_empty/')
# for item in os.listdir(main_path+'groups_of_64_nifti_non_empty/'):
  
#   print(item,sep,format_size(get_dir_size(main_path+'groups_of_64_nifti_non_empty/'+item)))
#   print('number of items',sep,len(os.listdir(main_path+'groups_of_64_nifti_non_empty/'+item)),'\n')

In [ ]:

# for files in os.listdir(main_path+'groups_of_64/imagesTs'):
#   try:
#     file_size = format_size(get_dir_size(main_path+'groups_of_64/imagesTs/'+files))
#     if file_size != '33.63 MB':
#       print(files,sep,file_size)

#   except NotADirectoryError:
#     continue

In [ ]:
##nifty to dicom slices
# nifty_folder = 'imagesTs'
# dcom_path = tspath
# the_file_name = 'liver_187.nii.gz'
# # for the_file_name in os.listdir(livpath+nifty_folder):
# if the_file_name.startswith('.') is not True:
#   clear()
#   print('checking',the_file_name,'\n')
#   nifty_file = livpath+nifty_folder+'/'+the_file_name
#   nifti_file = nibabel.load(nifty_file)
#   nifti_array = nifti_file.get_fdata()
#   number_slices = nifti_array.shape[2]
#   dcom_folder = dcom_path+'/'+the_file_name
#   dcom_file_number = len(os.listdir(dcom_folder))

#   # if number_slices != dcom_file_number:
#   print(the_file_name,number_slices,sep,dcom_file_number)
#   nifti2dicom_mfiles(nifty_file,dcom_path)
    

In [ ]:
# ##checking if the group_of_64_nifti has the same 64 slices or not
# folder = 'imagesTr'
# sixtyfournifties = livpath+folder+'/' 
# size_dict = {}
# for item in tqdm(os.listdir(sixtyfournifties)):
#   # print('checking',item)
#   the_file = sixtyfournifties+item
#   if item.startswith('.') is not True:
    
#     the_file = nibabel.load(the_file)
#     nifti_array = the_file.get_fdata()
#     number_slices = nifti_array.shape[2]
#     if number_slices not in size_dict:
#       size_dict[number_slices]=[item]
#     else:
#       size_dict[number_slices].append(item)
#   # if number_slices != 64:
#   #   clear()
#   #   print('this one has less',item)
#   #   break

#   # else:
#   #   clear()
#   #   print('\nsize',number_slices)
# size_dict

In [ ]:
# t = to_json(size_dict,'slice of files','read')
# k = []
# for item in t.keys():
#   k.append(int(item))

In [ ]:
# min(k),max(k)

In [ ]:
##slice 64 from the extracted dicoms into corresponding patient nifty
# group_folder = 'labelsTr'+'/'
# dcom_folder = labels+'/'

# g64 = main_path+'groups_of_64/'+group_folder


# the_file_name = 'liver_23.nii.gz'

# # for the_file_name in os.listdir(dcom_folder):
# clear()
# number = len(os.listdir(dcom_folder+the_file_name))
# the_name_of_the_patient = the_file_name.split('.')[0]
# cluster = []
# group_no = 0

# possible = int(number/64)
# print('for patient',the_name_of_the_patient,'\ntotal_file',number,'\npossible partitions',possible)

# for i in range(number):
#   # print(i)
  
#   cluster.append('slice'+str(i)+'.dcm')
#   if len(cluster)==64:
    
#     slices_to_folder(the_name_of_the_patient,group_no,cluster,g64,dcom_folder)
#     print(the_name_of_the_patient+'_'+str(group_no),sep,cluster)
#     cluster=[]
#     group_no+=1

# if cluster != []:
#   print('Remainder',len(cluster),sep,cluster)

# cache_easy['DATA_FOLD_'+sep+str(c)]={data_in}

# to_json(cache_easy,'cache_easy','write')

# device = torch.device("cuda")
# model = UNet(
#     dimensions=3,
#     in_channels=1,
#     out_channels=2,
#     channels=(16, 32, 64, 128, 256), 
#     strides=(2, 2, 2, 2),
#     num_res_units=2,
#     norm=Norm.BATCH,
# ).to(device)

# max_epochs = 600
# # test_interval = 1
# # #loss_function = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(1792651250,2510860).to(device))
# loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
# optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)
# train(model, data_in, loss_function, optimizer, epochs, model_dir)
